-Ingest and transform and split data

In [4]:

import random
import pathlib
import itertools
import collections

import os
import cv2
import numpy as np
import remotezip as rz
import tensorflow as tf

import imageio
from IPython import display
from urllib import request 
from tensorflow_docs.vis import embed



In [68]:
import moviepy.editor as mp

#we need to create list of all video files(done), and then label those video files(in progress), extract the .wav from each, then create training sets, 

In [33]:

#### function under construction ####

def dataDirectory(folder):
    newDirectory=[]
    for root, dirs, files in os.walk(folder):
        for file in files:
            if file.endswith('.mp4'):
                newDirectory.append(file)
    return newDirectory
    


In [32]:
len(RealFilesDirectory)

0

In [36]:

dataDirectory("rC:\\Users\\cumminan\\OneDrive - WWT\\Desktop\\RealVideo-RealAudio")

[]

In [69]:
FakeFilesDirectory=[]
FakeFilesNames=[]
fakeFileLocation=["C:\\Users\\cumminan\\RealVideo-FakeAudio"]
for root, dirs, files in os.walk(r"C:\\Users\\cumminan\\RealVideo-FakeAudio"):
    for file in files:
        if file.endswith('.mp4'):
            FakeFilesDirectory.append(os.path.join(root,file))
            FakeFilesNames.append(file)



In [70]:
RealFilesDirectory=[]
RealFilesNames=[]
realFileLocation=["C:\\Users\\cumminan\\RealVideo-FakeAudio"]
for root, dirs, files in os.walk(r"C:\\Users\\cumminan\\OneDrive - WWT\\Desktop\\RealVideo-RealAudio"):
    for file in files:
        if file.endswith('.mp4'):
            RealFilesDirectory.append(os.path.join(root,file))
            RealFilesNames.append(file)

In [67]:
FakeFilesDirectory[0]

'C:\\\\Users\\\\cumminan\\\\RealVideo-FakeAudio\\African\\men\\id00076\\00109_fake.mp4'

In [62]:
RealFilesDirectory[1]

'C:\\\\Users\\\\cumminan\\\\OneDrive - WWT\\\\Desktop\\\\RealVideo-RealAudio\\African\\men\\id00166\\00010.mp4'

In [72]:
#combine all video files
FRFilesDirect=FakeFilesDirectory+RealFilesDirectory

In [71]:
FRFilesNames=FakeFilesNames+RealFilesNames

In [66]:
FRFilesDirect[0]

'C:\\\\Users\\\\cumminan\\\\RealVideo-FakeAudio\\African\\men\\id00076\\00109_fake.mp4'

In [41]:
#create location for .wav files (necessary for data processing feeding into any model)

directory='wavFiles'
parent="C:\\Users\\cumminan"
path=os.path.join(parent,directory)
os.mkdir(path)


In [78]:
FRFilesNames[0]

'00109_fake.mp4'

In [ ]:
#convert files into tensorflow dataset 

In [83]:
WavFileNames=[]
for root, dirs, files in os.walk(r"C:\\Users\\cumminan\\wavFiles"):
    for file in files:
            WavFileNames.append(file)

In [84]:
len(WavFileNames)

356

In [85]:
def load_dataset(filenames):
    dataset= tf.data.Dataset.from_tensor_slices(filenames)
    return dataset

In [ ]:
load_dataset(WavFilesNames)

In [87]:
commands = np.array(tf.io.gfile.listdir(str("C:\\Users\\cumminan\\wavFiles")))
commands = commands[commands != 'README.md']

In [89]:
def decode_audio(audio_binary):
  audio, _ = tf.audio.decode_wav(audio_binary)
  return tf.squeeze(audio, axis=-1)

In [88]:
def get_label(filename):
  if 'fake' in filename:
    label=[1,0]
  else:
      label=[0,1]
  return label

In [ ]:
Commands=['fake','real']

In [90]:
def get_waveform_and_label(filename):
  label = get_label(filename)
  audio_binary = tf.io.read_file(filename)
  waveform = decode_audio(audio_binary)
  return waveform, label

In [91]:
def get_spectrogram(waveform, padding=False, min_padding=48000):
  waveform = tf.cast(waveform, tf.float32)
  spectrogram = tf.signal.stft(waveform, frame_length=2048, frame_step=512, fft_length=2048)
  spectrogram = tf.abs(spectrogram)
  return spectrogram

In [92]:
def get_spectrogram_tf(waveform, label):
  spectrogram = get_spectrogram(waveform)
  spectrogram = tf.expand_dims(spectrogram, axis=-1)
  return spectrogram, label

In [93]:
def prepare_sample(spectrogram, label):
  spectrogram = tf.image.resize(spectrogram, [HEIGHT, WIDTH])
  spectrogram = tf.image.grayscale_to_rgb(spectrogram)
  return spectrogram, label

In [94]:
HEIGHT, WIDTH = 128, 128
AUTO = tf.data.AUTOTUNE


In [96]:
def get_dataset(filenames, batch_size=32):
  dataset = load_dataset(filenames)
    
  dataset = files_ds.map(get_waveform_and_label, num_parallel_calls=AUTO)
  dataset = dataset.map(get_spectrogram_tf, num_parallel_calls=AUTO)
  dataset = dataset.map(prepare_sample, num_parallel_calls=AUTO)
  dataset = dataset.shuffle(256)
  dataset = dataset.repeat()
  dataset = dataset.batch(batch_size)
  dataset = dataset.prefetch(AUTO)
  return dataset

In [97]:
def model_fn(input_shape, N_CLASSES):
  inputs = L.Input(shape=input_shape, name='input_audio')
  base_model = efn.EfficientNetB0(input_tensor=inputs, 
                                  include_top=False, 
                                  weights='imagenet')

  x = L.GlobalAveragePooling2D()(base_model.output)
  x = L.Dropout(.5)(x)
  output = L.Dense(N_CLASSES, activation='softmax',name='output')(x)
    
  model = Model(inputs=inputs, outputs=output)

  return model

In [73]:
for i in range(0,len(FRFilesDirect)):
    fileSource=FRFilesDirect[i]
    clip=mp.VideoFileClip(fileSource)
    clip.audio.write_audiofile("C:\\Users\\cumminan\\wavFiles"+FRFilesNames[i]+'.wav')
    

MoviePy - Writing audio in C:\Users\cumminan\wavFiles00109_fake.mp4.wav


MoviePy - Done.
MoviePy - Writing audio in C:\Users\cumminan\wavFiles00010_fake.mp4.wav


MoviePy - Done.
MoviePy - Writing audio in C:\Users\cumminan\wavFiles00118_fake.mp4.wav


MoviePy - Done.
MoviePy - Writing audio in C:\Users\cumminan\wavFiles00118_fake.mp4.wav


MoviePy - Done.
MoviePy - Writing audio in C:\Users\cumminan\wavFiles00052_fake.mp4.wav


...

MoviePy - Done.
MoviePy - Writing audio in C:\Users\cumminan\wavFiles00093.mp4.wav


MoviePy - Done.
MoviePy - Writing audio in C:\Users\cumminan\wavFiles00028.mp4.wav


MoviePy - Done.
MoviePy - Writing audio in C:\Users\cumminan\wavFiles00092.mp4.wav


MoviePy - Done.
MoviePy - Writing audio in C:\Users\cumminan\wavFiles00021.mp4.wav


MoviePy - Done.


In [ ]:
#visually compare fake audio vs real audio for two video clips



#source information
fakeSampleLoc='C:\\Users\\cumminan\\SampleAudioFiles\\Fake\\00280_fake.mp4'
realSampleLoc='C:\\Users\\cumminan\\SampleAudioFiles\\Real\\00280.mp4'
fakeClip=mp.VideoFileClip(fakeSampleLoc)
realClip=mp.VideoFileClip(realSampleLoc)


#extract audio

fakeClip.audio.write_audiofile("C:\\Users\\cumminan\wavFiles"+i+'.wav')
realClip.audio.write_audiofile(r"realClip.wav")

In [ ]:
#key things, tensorflow dataset type


In [11]:
files = list_files_from_zip_url(filesource1)
files = [f for f in files if f.endswith('.mp4')]
files[:10]

RemoteIOError: No connection adapters were found for 'C:\\Users\\cumminan\\Downloads\\FakeAVCeleb_v1.2'